In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
bearer_token = os.environ.get("bearer-token")

In [3]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAHalVAEAAAAAVViwLGyKNUuT6ssD8tDTqiyQ1Pw%3DVXK9D7qXXABAcb96CPXxbltVZy3C9mzONePX8nLr9mkN3oHa7u'

In [4]:
def auth():
    return os.environ.get("TOKEN")

In [5]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [6]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [7]:
def create_url(keyword, start_date, end_date, next_token = None):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 
                    'start_time' : start_time, 
                    'end_time' : end_time,
                    'max_results' : max_results,
                    'tweet.fields': 'created_at',
                    'next_token': {}
                     }
    
    return (search_url, query_params)

In [8]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.get( url, auth=bearer_oauth, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [9]:
#Inputs for the request
#bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "amazon lang:en"
start_time = "2021-11-01T21:06:00.000Z"
end_time = "2021-11-03T21:27:53.241Z"
max_results = 10
next_token = None

In [10]:
url = create_url(keyword, start_time,end_time)


In [11]:
json_response = connect_to_endpoint(url[0], headers, url[1], next_token)

Endpoint Response Code: 200


In [12]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "created_at": "2021-11-03T21:27:52.000Z",
            "id": "1456010264046936065",
            "text": "@jeb2344 \n\nLISTEN NOW! #MotivationalWisdom Episodes has shifted up to Amazon Alexa. Customer Access: \"Alexa, play the podcast #MotivationalWisdom\" Lady Cynthia Renee Kelly. Select the specific episode from the list below. \n\nhttps://t.co/vOzeBSN6zr https://t.co/e49oEfTHtQ"
        },
        {
            "created_at": "2021-11-03T21:27:51.000Z",
            "id": "1456010256086179843",
            "text": "Ilya Ivashka vs Daniil Medvedev is on Amazon Prime Video. With commentary from Pete Odgers. Co-commentator is Annabel Croft.  For more details see https://t.co/m4h15Ex6hP #IlyaIvashka #DaniilMedvedev #AmazonPrimeVideo"
        },
        {
            "created_at": "2021-11-03T21:27:51.000Z",
            "id": "1456010255838687239",
            "text": "$WISH where does small business go to sell product in a marketplace that allows them r

In [13]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['created_at','id','tweet'])
csvFile.close()

In [14]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        #Time created
        created_at = dateutil.parser.parse(tweet['created_at'])


        #Tweet ID
        tweet_id = tweet['id']

        #Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [created_at, tweet_id, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [15]:
append_to_csv(json_response, "data.csv")

# of Tweets added from this response:  10


In [16]:
int(json_response.headers["x-rate-limit-remaining"])

AttributeError: 'dict' object has no attribute 'headers'